In [ ]:
from transformers import BertPreTrainedModel, BertModel
from transformers.modeling_outputs import SequenceClassifierOutput
import torch.nn as nn
import losses

class BertScratch(BertPreTrainedModel):
    def __init__(self, config):
        
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config
        self.alpha = 0.2

        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.post_init()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids, attention_mask, token_type_ids)
        # (batch_size, hidden_size)
        pooled_output = outputs[1]
    
        pooled_output = self.dropout(pooled_output)
        # (batch_size, num_labels)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            ce_loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            
            scl_fct = losses.SupConLoss()
            scl_loss = scl_fct(pooled_output, labels)

            loss = ce_loss + self.alpha * scl_loss

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions
        )


## 自定义模型

- 需要继承自PreTrainedModel, 在init函数中向父类传递参数，在forward函数中进行计算
- 若结果存在label，那么需要重写forward方法来计算每个逻辑的损失

## BertModel的输出

1. ​​BERT 模型的输出结构​​

BERT 模型的输出是一个元组（或 BaseModelOutput 对象），包含以下内容：
- ​outputs[0]​​: 所有 token 的隐藏状态（形状为 (batch_size, sequence_length, hidden_size)），即每个 token 的上下文表示。
- ​outputs[1]​​: 池化后的序列表示（形状为 (batch_size, hidden_size)），通常对应 [CLS] token 的隐藏状态经过额外线性层和激活函数（如 tanh）处理后的结果，用于分类任务。
- ​​outputs.hidden_states​​: 所有层的隐藏状态（需设置 output_hidden_states=True）
- outputs.attentions​​: 注意力权重（需设置 output_attentions=True）。

2. ​​为什么使用 outputs[1]？​​
​
- ​分类任务需求​​：在序列分类任务中，通常需要将整个序列的信息压缩为一个固定长度的向量。BERT 的 
[CLS] token 的池化输出（outputs[1]）被设计为捕获整个序列的全局信息。
- ​与 outputs[0] 的区别​​：
  - outputs[0] 包含所有 token 的细粒度表示，适合 token 级任务（如 NER）。
  - outputs[1] 是聚合后的表示，适合序列级任务（如文本分类）。

3. ​​代码中的具体应用​​
在 BertScratch 中：
```python
pooled_output = outputs[1]  # 提取池化后的表示 (batch_size, hidden_size)
pooled_output = self.dropout(pooled_output)
logits = self.classifier(pooled_output)  # 分类层输入
```
- ​pooled_output​​ 作为分类器的输入，通过线性层 (self.classifier) 映射到标签空间。

**总结**

outputs[1] 是 BERT 为分类任务设计的池化输出，形状为 (batch_size, hidden_size)，而非 token 级的 (batch_size, sequence_length, hidden_size)。这种设计简化了序列级任务的流程

## loss相关

### CrossEntropyLoss

交叉熵（Cross Entropy）损失是在分类问题中常用的损失函数，尤其在神经网络的训练中经常被使用。它衡量了模型的预测概率分布与实际标签的分布之间的差异

二分类交叉熵公式
$$
L = -\frac{1}{N} \sum_{i=1}^N \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]
$$
C分类交叉熵公式
$$
L = -\frac{1}{N} \sum_{i=1}^N \sum_{j=1}^C y_{i,j} \log(\hat{y}_{i,j})
$$
$y_{i,j}$是one-hot编码

In [ ]:
loss_fct = nn.CrossEntropyLoss()
ce_loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

1. ​​nn.CrossEntropyLoss() 的功能​​
- ​作用​​：
  该损失函数结合了 Softmax 和负对数似然损失（NLLLoss），用于衡量模型预测的概率分布与真实标签的差异，适用于多分类任务。
- ​​输入要求​​：
  - ​logits​​：模型的原始输出（未经过 Softmax），形状需为 (batch_size, num_classes）。
  - labels​​：真实标签，形状为 (batch_size,)，每个元素是类别的整数索引（如 [0, 2, 1]）。

注意，输入要求为logits而不能经过Softmax，形状为（batch_size, num_classes）

2. 为什么需要对logits和labels的形状进行调整？
- 输入要求​​：nn.CrossEntropyLoss要求logits形状为(N, C)（N是样本数，C是类别数），标签形状为(N,)

- 三维logits的场景​​：在token级任务（如命名实体识别、token级情感分析）中，模型对每个token输出分类结果，此时logits形状为(batch_size, seq_len, num_labels)。展平后：
  - ​​logits​​：(batch_size * seq_len, num_labels)
  - ​labels​​：(batch_size * seq_len,)


### SupConLoss

In [ ]:
"""
Author: Yonglong Tian (yonglong@mit.edu)
Date: May 07, 2020
"""
from __future__ import print_function

import torch
import torch.nn as nn


class SupConLoss(nn.Module):
    """Supervised Contrastive Learning: https://arxiv.org/pdf/2004.11362.pdf.
    It also supports the unsupervised contrastive loss in SimCLR"""
    def __init__(self, temperature=0.07, contrast_mode='all',
                 base_temperature=0.07):
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.contrast_mode = contrast_mode
        self.base_temperature = base_temperature

    def forward(self, features, labels=None, mask=None):
        """Compute loss for model. If both `labels` and `mask` are None,
        it degenerates to SimCLR unsupervised loss:
        https://arxiv.org/pdf/2002.05709.pdf

        Args:
            features: hidden vector of shape [bsz, n_views, ...].
            labels: ground truth of shape [bsz].
            mask: contrastive mask of shape [bsz, bsz], mask_{i,j}=1 if sample j
                has the same class as sample i. Can be asymmetric.
        Returns:
            A loss scalar.
        """
        
        device = (torch.device('cuda')
                  if features.is_cuda
                  else torch.device('cpu'))

        features = features.view(features.shape[0], features.shape[1], -1)

        batch_size = features.shape[0]
        """ 
            构建mask: mask[i,j] = 1表示两者属于同一label
        """
        if labels is not None and mask is not None:
            raise ValueError('Cannot define both `labels` and `mask`')
        elif labels is None and mask is None:
            mask = torch.eye(batch_size, dtype=torch.float32).to(device)
        elif labels is not None:
            labels = labels.contiguous().view(-1, 1)
            if labels.shape[0] != batch_size:
                raise ValueError('Num of labels does not match num of features')
            mask = torch.eq(labels, labels.T).float().to(device)
        else:
            mask = mask.float().to(device)
        """
            将feature从(batch_size, n_views, hidden_size)转换为(batch_size*n_view, hidden_size)
        """
        contrast_count = features.shape[1]
        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
        if self.contrast_mode == 'one':
            anchor_feature = features[:, 0]
            anchor_count = 1
        elif self.contrast_mode == 'all':
            anchor_feature = contrast_feature
            anchor_count = contrast_count
        else:
            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))

        # compute logits
        anchor_dot_contrast = torch.div(
            torch.matmul(anchor_feature, contrast_feature.T),
            self.temperature)
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()

        # tile mask
        mask = mask.repeat(anchor_count, contrast_count)
        # mask-out self-contrast cases
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * anchor_count).view(-1, 1).to(device),
            0
        )
        mask = mask * logits_mask

        # compute log_prob
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)

        # loss
        loss = - (self.temperature / self.base_temperature) * mean_log_prob_pos
        loss = loss.view(anchor_count, batch_size).mean()

        return loss

#### SupConLoss的思想：
- 将无监督学习中的对比学习拓展到有监督学习中来：在无监督学习中每个样例只有一个正样本，和其它2N-1个负样本，如果其他2N-1个样本中如果有和目前样本同一类别的样本，也会被视为负样本。在有监督学习的对比学习中：将同属一个类别的视为正样本，其他的为负样本。

### 代码中相关函数小节：
- torch.mean(input,*): 返回所有元素的平均值  
- torch.eq(input,other): 判断每个相应的元素是否相等，注意第二个参数可以广播
- torch.unbind(input, dim=0): 移除张量的一个维度。返回给定维度上所有已去除该维度的切片的元组。
  - 假设a.shape = (3,4), 则torch.unbind(a)返回(a[0], a[1], a[2])
- torch.cat(tensors, dim=0, *, out=None) → Tensor:在给定维度上连接给定的张量序列。所有张量要么具有相同的形状（除了连接维度），要么是大小为 (0,) 的一维空张量。
  - cat不会增加新的维度，但会修改指定的维度，stack会添加新维度